# 开始实战

In [1]:
%store -r train_housing_labels
%store -r train_housing_prepared

### 7、选择及训练模型

#### 线性回归模型(LinearRegression)

In [2]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(train_housing_prepared, train_housing_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

训练完成，然后评估模型，计算训练集中的均方根误差(RMSE)

In [3]:
import numpy as np
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(train_housing_prepared)
lin_mse = mean_squared_error(train_housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68628.19819848922

#### 决策树模型(DecisionTreeRegressor)

In [4]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(train_housing_prepared, train_housing_labels)
housing_predictions = tree_reg.predict(train_housing_prepared)
tree_mse = mean_squared_error(train_housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

可以看到决策树回归模型的的训练集均方误差竟然为0。比线性回归模型的的训练集均方误差小太多太多。  
但这是否说明了决策树回归模型比线性回归模型在此问题上好很多，当然不是，训练误差小的模型并不代表为好模型，这是因为模型可能过度地学习了训练集的数据，只是在训练集上的表现好（即过拟合），一旦测试新的数据表现就会很差。  
因此在训练的时候需要将部分的训练数据提取出来作为验证集，验证该模型是否对此问题适用。其中比较常用的就是交叉验证法。 

#### 交叉验证法

在Scikit-Learn中交叉验证对应的类为cross_val_score，下面是线性回归模型与决策树回归模型的交叉验证实例：

In [5]:
from sklearn.model_selection import cross_val_score
tree_scores = cross_val_score(tree_reg, train_housing_prepared, train_housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_scores = cross_val_score(lin_reg, train_housing_prepared, train_housing_labels,
                            scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-tree_scores)
lin_rmse_scores = np.sqrt(-lin_scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    
display_scores(tree_rmse_scores)
display_scores(lin_rmse_scores)

Scores: [69499.81023221 65161.50251187 69972.80744939 69492.85810585
 70111.41744402 74876.80242351 69202.02049344 69861.00307739
 77374.76793778 69469.79369411]
Mean: 70502.27833695686
Standard deviation: 3170.431718700456
Scores: [66782.73843989 66960.118071   70347.95244419 74739.57052552
 68031.13388938 71193.84183426 64969.63056405 68281.61137997
 71552.91566558 67665.10082067]
Mean: 69052.46136345083
Standard deviation: 2731.674001798348


可以看到决策树回归模型的交叉验证平均误差为71163，而线性回归模型的交叉验证平均误差为69051，这说明决策树回归模型明显是过拟合，实际上比线性回归模型要差一些。  
除了这两个简单的模型以外，还应该试验不同的模型（如随机森林，不同核的SVM，神经网络等），最终选择2-5个候选的模型。（也可以写到同一个文件下，方便以后直接调用）

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### 保存模型

In [5]:
from sklearn.externals import joblib
joblib.dump(lin_reg, "lin-reg.pkl")
# and later...
#my_model_loaded = joblib.load("my_model.pkl")  加载模型

['lin-reg.pkl']

## 8、模型调参

#### 网格搜索 Gird Search

scikit-learn中提供函数GridSearchCV用于网格搜索调参，网格搜索就是通过自己对模型需要调整的几个参数设定一些可行值，然后Grid Search会排列组合这些参数值，每一种情况都去训练一个模型，经过交叉验证后输出结果。下面为随机森林回归模型（RandomForestRegression）的一个Grid Search的例子。

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap':[False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error')
grid_search.fit(train_housing_prepared, train_housing_labels)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [7]:
grid_search.best_params_

{'max_features': 6, 'n_estimators': 30}

In [8]:
cvres = grid_search.cv_results_

In [9]:
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

63955.35510303105 {'max_features': 2, 'n_estimators': 3}
55475.68987952954 {'max_features': 2, 'n_estimators': 10}
52819.46111111589 {'max_features': 2, 'n_estimators': 30}
60606.840633316504 {'max_features': 4, 'n_estimators': 3}
52758.28882845016 {'max_features': 4, 'n_estimators': 10}
50485.29752802271 {'max_features': 4, 'n_estimators': 30}
60296.568561361586 {'max_features': 6, 'n_estimators': 3}
51956.25542449312 {'max_features': 6, 'n_estimators': 10}
49779.1738644514 {'max_features': 6, 'n_estimators': 30}
58965.61278624701 {'max_features': 8, 'n_estimators': 3}
51911.13481182861 {'max_features': 8, 'n_estimators': 10}
49879.13888652553 {'max_features': 8, 'n_estimators': 30}
62755.36540738579 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
53777.31647022583 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59237.06772276107 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52261.489508786966 {'bootstrap': False, 'max_features': 3, 'n_estimators': 

#### 随机搜索(Randomized Search)

由于上面的网格搜索搜索空间太大，而机器计算能力不足，则可以通过给参数设定一定的范围，在范围内使用随机搜索选择参数，随机搜索的好处是能在更大的范围内进行搜索，并且可以通过设定迭代次数n_iter，根据机器的计算能力来确定参数组合的个数，下面给出一个随机搜索的例子。

In [11]:
from sklearn.model_selection import RandomizedSearchCV
params_ran = {'n_estimators':range(30,50), 'max_features':range(3, 8)}
forest_reg = RandomForestRegressor()
random_search = RandomizedSearchCV(forest_reg, params_ran, cv=5, scoring='neg_mean_squared_error',
                                  n_iter=10)
random_search.fit(train_housing_prepared, train_housing_labels)

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'n_estimators': range(30, 50), 'max_features': range(3, 8)},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='neg_mean_squared_error',
          verbose=0)

In [12]:
random_search.best_params_

{'n_estimators': 49, 'max_features': 7}

#### 分析最好的模型每个特征的重要性

假设现在调参以后得到最好的参数模型，然后可以查看每个特征对预测结果的贡献程度，根据贡献程度，可以删减减少一些不必要的特征。

In [13]:
%store -r encoder
%store -r num_attribs
feature_importances = grid_search.best_estimator_.feature_importances_
extra_attribs = ["room_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_one_hot_attribs = list(encoder.classes_)
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.33111775691575007, 'median_income'),
 (0.15779179306623528, 'INLAND'),
 (0.10806737363462225, 'pop_per_hhold'),
 (0.07721470879523552, 'bedrooms_per_room'),
 (0.07309837070828527, 'longitude'),
 (0.06517323030464922, 'latitude'),
 (0.05515112576791306, 'room_per_hhold'),
 (0.04107282571105731, 'housing_median_age'),
 (0.019067582391249838, 'total_rooms'),
 (0.01811480283104651, 'population'),
 (0.01697624638885219, 'total_bedrooms'),
 (0.016617755030718193, 'households'),
 (0.01098864332202041, '<1H OCEAN'),
 (0.005558355938433598, 'NEAR OCEAN'),
 (0.003892456315134704, 'NEAR BAY'),
 (9.69728787965914e-05, 'ISLAND')]

In [21]:
%store -r X_test_prepared
%store -r y_test

#### 在测试集中评估

In [23]:
final_model = grid_search.best_estimator_
final_predictions = final_model.predict(X_test_prepared)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

47822.52472534212

可以发现，结果和交叉验证以后的结果比较相似，说明经过交叉验证后，在新的数据集上也能达到类似的效果。  
需要注意：在测试集中补缺失值，标准化等用到的值都是训练集上的中值，平均值等，而不是测试集上的。因为必须把数据放缩到同一尺度。